In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent/HOTS


In [2]:
import os
import tonic
import numpy as np
#from mix_Network import *
download = False
learn_set = tonic.datasets.NMNIST(save_to='../Data/',
                                  train=True, download=download,
                                  transform=tonic.transforms.AERtoVector()
                                 )

In [3]:
records_path = '../Records'
import datetime
timestr = datetime.datetime.now().date().isoformat()
timestr = '2021-02-13'
verbose = True

%mkdir -p ../Records
%mkdir -p ../Records/EXP_03_NMNIST

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 5
nb_train = int(len(learn_set)//ds)
print(f'The dataset has size {len(learn_set)}, using {nb_train}')

The dataset has size 60000, using 12000


In [4]:
import time
def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
# https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html
#criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size
criterion = torch.nn.BCELoss(reduction="mean") # loss divided by output size

class LogisticRegressionModel(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LogisticRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.Softmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [6]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 5 + 1
#num_epochs = 2 ** 9 + 1
num_epochs

33

In [7]:
# batch_size = 256
n_classes=10
amsgrad = False # gives similar results
amsgrad = True  # gives similar results

def fit_raw_data(dataset, 
            nb_digit,
            learning_rate=learning_rate,
            # batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, #**kwargs
        ):
    

    generator = torch.Generator().manual_seed(42)
    sampler = torch.utils.data.RandomSampler(dataset, replacement=True, num_samples=nb_digit, generator=generator)
    loader = tonic.datasets.DataLoader(dataset, sampler=sampler)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')

    N = 34*34*2
    n_classes = 10
    logistic_model = LogisticRegressionModel(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    
    for epoch in range(int(num_epochs)):
        losses = []
        for X, label in loader:
            X, label = X.to(device), label.to(device)
            X, label = X.squeeze(0), label.squeeze(0) # just one digit = one batch
            outputs = logistic_model(X)

            n_events = X.shape[0]
            #print(X.squeeze(0).shape, label * torch.ones((1, n_events)))
            #print(outputs, label)
            labels = label*torch.ones(n_events).type(torch.LongTensor).to(device)
            labels = torch.nn.functional.one_hot(labels, num_classes=n_classes).type(torch.DoubleTensor).to(device)
            #print(outputs.shape, labels.shape)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")
            
    return logistic_model, losses

In [8]:
tic()
model, loss = fit_raw_data(learn_set, 
            nb_train,
            learning_rate=learning_rate,
            # batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=True,
        )
toc()

device -> cpu
Iteration: 0 - Loss: 0.26714
Iteration: 1 - Loss: 0.24016
Iteration: 2 - Loss: 0.23863
Iteration: 3 - Loss: 0.24644
Iteration: 4 - Loss: 0.23490
Iteration: 5 - Loss: 0.22981
Iteration: 6 - Loss: 0.23316
Iteration: 7 - Loss: 0.22924
Iteration: 8 - Loss: 0.23376
Iteration: 9 - Loss: 0.22584
Iteration: 10 - Loss: 0.22956
Iteration: 11 - Loss: 0.23152
Iteration: 12 - Loss: 0.22968
Iteration: 13 - Loss: 0.22783
Iteration: 14 - Loss: 0.22772
Iteration: 15 - Loss: 0.22450
Iteration: 16 - Loss: 0.22462
Iteration: 17 - Loss: 0.23698
Iteration: 18 - Loss: 0.23182
Iteration: 19 - Loss: 0.22715
Iteration: 20 - Loss: 0.23062
Iteration: 21 - Loss: 0.23459
Iteration: 22 - Loss: 0.22609
Iteration: 23 - Loss: 0.22445
Iteration: 24 - Loss: 0.23015
Iteration: 25 - Loss: 0.23091
Iteration: 26 - Loss: 0.22291
Iteration: 27 - Loss: 0.22773
Iteration: 28 - Loss: 0.21864
Iteration: 29 - Loss: 0.22755
Iteration: 30 - Loss: 0.23208
Iteration: 31 - Loss: 0.22348
Iteration: 32 - Loss: 0.22112
Done i

In [9]:
test_set = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False, download=download,
                                transform=tonic.transforms.AERtoVector())

In [10]:
nb_test = int(len(test_set)//ds)
print(f'The dataset has size {len(test_set)}, using {nb_test}')

The dataset has size 10000, using 2000


In [11]:
def predict_data(test_set, model, # gamma=gamma,
            verbose=False, **kwargs
        ):
    
    with torch.no_grad():

        generator=torch.Generator().manual_seed(42)
        sampler = torch.utils.data.RandomSampler(test_set, replacement=True, num_samples=nb_test, generator=generator)
        loader = tonic.datasets.DataLoader(test_set, sampler=sampler)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        logistic_model = model.to(device)

        pred_target, true_target = [], []

        for X, label in loader:
            X = X.to(device)
            X, label = X.squeeze(0), label.squeeze(0)

            n_events = X.shape[0]
            labels = label*torch.ones(n_events).type(torch.LongTensor)

            outputs = logistic_model(X)

            pred_target.append(torch.argmax(outputs, dim=1).cpu().numpy())
            true_target.append(labels.numpy())

    return pred_target, true_target

In [12]:
pred_target, true_target = predict_data(test_set, model)

In [13]:
accuracy = []
for pred_target_, true_target_ in zip(pred_target, true_target):
    accuracy.append(np.mean(pred_target_ == true_target_))
print(f'{np.mean(accuracy)=:.3f}')    

np.mean(accuracy)=0.738
